In [4]:
import tkinter as tk
from tkinter import filedialog,Canvas,Scrollbar,simpledialog
from pdf2image import convert_from_path
from PIL import Image, ImageTk,ImageDraw,ImageFont
import json
import os

In [5]:
class PDFImageApp:
    def __init__(self, root):
        root.title("PDF to Image Viewer")
        root.geometry("1920x1080")
        
        self.instruction_label = tk.Label(root, text="Upload a PDF to start.", font=("Arial", 16))
        self.instruction_label.place(x=700, y=30)
        
        self.upload_pdf_button = tk.Button(root, text="Upload PDF", command=self.upload_pdf, font=("Arial", 14))
        self.upload_pdf_button.place(x=750, y=60)
        
        self.canvas = tk.Canvas(root, bg="white", width=1400, height=850)
        self.canvas.place(x=100, y=100)
        
        self.prev_button = tk.Button(root, text="<< Previous", command=self.prev_image, font=("Arial", 14), state=tk.DISABLED)
        self.prev_button.place(x=200, y=960)
        
        self.next_button = tk.Button(root, text="Next >>", command=self.next_image, font=("Arial", 14), state=tk.DISABLED)
        self.next_button.place(x=1200, y=960)

        self.apply_format_button = tk.Button(root, text="Apply Formatting", command=self.apply_formatting, font=("Arial", 14))
        self.apply_format_button.place(x=1400, y=960)
        
        self.upload_json_button = tk.Button(root, text="Upload JSON", command=self.upload_json, font=("Arial", 14), state=tk.DISABLED)
        self.upload_json_button.place(x=700, y=960)
        
        self.scroll_frame = tk.Frame(root)
        self.scroll_frame.place(x=100, y=100, width=1600, height=850)

        self.v_scroll = Scrollbar(self.scroll_frame, orient=tk.VERTICAL)
        self.v_scroll.pack(side=tk.RIGHT, fill=tk.Y)

        self.h_scroll = Scrollbar(self.scroll_frame, orient=tk.HORIZONTAL)
        self.h_scroll.pack(side=tk.BOTTOM, fill=tk.X)

        self.canvas = Canvas(self.scroll_frame, bg="white", width=1600, height=850,
                             yscrollcommand=self.v_scroll.set, xscrollcommand=self.h_scroll.set)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.v_scroll.config(command=self.canvas.yview)
        self.h_scroll.config(command=self.canvas.xview)


        self.save_button = tk.Button(root, text="Save", command=self.save_data, font=("Arial", 14))
        self.save_button.place(x=500, y=960)


        self.images = []
        self.current_image_index = 0
        self.start_x = None
        self.start_y = None
        self.current_rectangle = None
        self.bounding_boxes = {}

        self.canvas.bind("<Button-1>", self.start_draw_bbox)
        self.canvas.bind("<B1-Motion>", self.dragging)
        self.canvas.bind("<ButtonRelease-1>", self.end_draw_bbox)


    def save_data(self):
        dirpath = filedialog.askdirectory(title="Select a Directory to Save Images and JSON")
        if not dirpath: return
        
        # Save original images
        for idx, img in enumerate(self.images):
            img_path = os.path.join(dirpath, f"original_image_{idx}.png")
            img.save(img_path, 'PNG')
        
        # Create the JSON data
        json_data = {
            'bounding_boxes': self.bounding_boxes
        }
        
        # Save the JSON data
        json_path = os.path.join(dirpath, "bounding_boxes.json")
        with open(json_path, 'w') as f:
            json.dump(json_data, f, indent=4)

    def apply_formatting(self):
        dirpath = filedialog.askdirectory(title="Select a Directory to Save PDFs")
        if not dirpath: return

        max_texts = max([len(formatting['texts']) for formatting in self.bbox_formatting.values()])

        for text_idx in range(max_texts):
            modified_images = [img.copy() for img in self.images]

            for name, bbox_data_list in self.bounding_boxes.items():
                formatting = self.bbox_formatting.get(name, {})
                texts = formatting.get("texts", [])
                if text_idx >= len(texts): continue
                text = texts[text_idx]
                for bbox_data in bbox_data_list:
                    img = modified_images[bbox_data["image_index"]]
                    draw = ImageDraw.Draw(img)
                    draw.rectangle(bbox_data['coords'], fill=formatting.get("background_color"))
                    font_size = formatting.get("font", {}).get("size", 20)
                    font_color = formatting.get("font", {}).get("color", "black")
                    font_style = formatting.get("font", {}).get("style", "normal")
                    
                    if font_style == "bold":
                            font = ImageFont.truetype("d:/times new roman bold.ttf", font_size)
                    elif font_style == "italic":
                            font = ImageFont.truetype("d:/times new roman italic.ttf", font_size)
                    elif font_style == "default":
                            font=ImageFont.truetype("d:/noto/NotoSans-Bold.ttf")
                    else:
                            font = ImageFont.truetype("d:/times new roman.ttf", font_size)
                    text_x = bbox_data['coords'][0]
                    text_y = bbox_data['coords'][1]
                    draw.text((text_x, text_y), text, fill=font_color, font=font)

            pdf_filename = f"modified_{text_idx + 1}.pdf"
            pdf_path = os.path.join(dirpath, pdf_filename)
            modified_images[0].save(pdf_path, save_all=True, append_images=modified_images[1:])

        self.display_image(self.current_image_index)

    def start_draw_bbox(self, event):
        self.canvas.delete(self.current_rectangle)
        self.current_rectangle = None
        self.start_x = self.canvas.canvasx(event.x)
        self.start_y = self.canvas.canvasy(event.y)

    def dragging(self, event):
        x = self.canvas.canvasx(event.x)
        y = self.canvas.canvasy(event.y)
        if self.current_rectangle:
            self.canvas.delete(self.current_rectangle)
        self.current_rectangle = self.canvas.create_rectangle(self.start_x, self.start_y, x, y, outline='red')

    def end_draw_bbox(self, event):
        bbox_name = simpledialog.askstring("Bounding Box Name", "Name For Field:")
        if bbox_name:
            bbox_data = {
                "coords": (self.start_x, self.start_y, self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)),
                "image_index": self.current_image_index
            }
            if bbox_name in self.bounding_boxes:
                self.bounding_boxes[bbox_name].append(bbox_data)
            else:
                self.bounding_boxes[bbox_name] = [bbox_data]

    def upload_pdf(self):
        filepath = filedialog.askopenfilename(title="Select a PDF", filetypes=[("PDF files", "*.pdf")])
        if not filepath: return
        self.images = convert_from_path(filepath)
        self.display_image(0)
        if len(self.images) > 1:
            self.next_button.config(state=tk.NORMAL)
        self.upload_json_button.config(state=tk.NORMAL)

    def display_image(self, index):
        self.canvas.delete("all")
        self.tk_image = ImageTk.PhotoImage(self.images[index])
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
        self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))

    def prev_image(self):
        if self.current_image_index > 0:
            self.current_image_index -= 1
            self.display_image(self.current_image_index)
            if self.current_image_index == 0:
                self.prev_button.config(state=tk.DISABLED)
        self.next_button.config(state=tk.NORMAL)

    def next_image(self):
        if self.current_image_index < len(self.images) - 1:
            self.current_image_index += 1
            self.display_image(self.current_image_index)
            if self.current_image_index == len(self.images) - 1:
                self.next_button.config(state=tk.DISABLED)
        self.prev_button.config(state=tk.NORMAL)

    def upload_json(self):
        filepath = filedialog.askopenfilename(title="Select a JSON", filetypes=[("JSON files", "*.json")])
        if not filepath: return
        with open(filepath, 'r') as f:
            self.bbox_formatting = json.load(f)


In [6]:
root = tk.Tk()
app = PDFImageApp(root)
root.mainloop()